In [13]:
# FINDING INTERSECTION AND COMMON COLUMNS, TAKING OUT IMPORTANT COLUMNS OUT OF INTERSECTION

import pandas as pd
import numpy as np

df17 = pd.read_csv(r"raw\2017.csv")
df18 = pd.read_csv(r"raw\2018.csv")
df19 = pd.read_csv(r"raw\2019.csv")
df20 = pd.read_csv(r"raw\2020.csv")
df21 = pd.read_csv(r"raw\2021.csv")

cols = {}

for year, df in zip(
    range(2017, 2022),
    [df17, df18, df19, df20, df21]
):
    cols[year] = set(df.columns)
    
common_cols = set.intersection(*cols.values())

important_cols = {
    # Workplace support & benefits
    "Does your employer provide mental health benefits as part of healthcare coverage?", #1
    "Do you know the options for mental health care available under your employer-provided health coverage?",
    "Does your employer offer resources to learn more about mental health disorders and options for seeking help?",
    "Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?",
    "Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",
    "Overall, how much importance does your employer place on mental health?",
    "Overall, how much importance does your employer place on physical health?",
    "Do you know local or online resources to seek help for a mental health issue?",

    # Stigma, disclosure & comfort
    "Would you feel comfortable discussing a mental health issue with your direct supervisor(s)?", #2
    "Would you feel comfortable discussing a mental health issue with your coworkers?",
    "Would you feel more comfortable talking to your coworkers about your physical health or your mental health?",
    "Have you ever discussed your mental health with your employer?",
    "Have you ever discussed your mental health with coworkers?",
    "Have your observations of how another individual who discussed a mental health issue made you less likely to reveal a mental health issue yourself in your current workplace?",

    # Impact on work & treatment
    "Do you believe your productivity is ever affected by a mental health issue?", #3
    "If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?",
    "Have you ever sought treatment for a mental health disorder from a mental health professional?",
    "Do you have a family history of mental illness?",
    "Have you had a mental health disorder in the past?",
    "Do you have medical coverage (private insurance or state-provided) that includes treatment of mental health disorders?",

    # Company context & demographics
    "How many employees does your company or organization have?", #4
    "Is your primary role within your company related to tech/IT?",
    "Is your employer primarily a tech company/organization?",
    "What is your age?",
    "What is your gender?",
    "What is your race?",
}

print(len(common_cols))
print(len(important_cols))

ndf_17 = df17[[c for c in df17 if c in important_cols]]
ndf_17.to_csv('processed/n17.csv')

ndf_18 = df18[[c for c in df18 if c in important_cols]]
ndf_18.to_csv('processed/n18.csv')

ndf_19 = df19[[c for c in df19 if c in important_cols]]
ndf_19.to_csv('processed/n19.csv')

ndf_20 = df20[[c for c in df20 if c in important_cols]]
ndf_20.to_csv('processed/n20.csv')

ndf_21 = df21[[c for c in df21 if c in important_cols]]
ndf_21.to_csv('processed/n21.csv')



59
26


In [14]:
# NORMALISING COLUMNS AND CLEANING VALUES, MAINTAINING CONSISTENCY OF VALUES

import pandas as pd
import re 

def renaming_col(df):
    df.rename(columns= {"Would you feel more comfortable talking to your coworkers about your physical health or your mental health?": "you_feel_more_comfortable_talking_to_your_coworkers_about_your_physical_health_or_your_mental_health"}, inplace = True)
    return df

def normalize_columns(df):
    df = df.copy()
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(r"<.*?>", "", regex=True)   # remove html tags
        .str.replace(r"[^a-z0-9]+", "_", regex=True)  # non-alphanumerics → _
        .str.replace(r"_+", "_", regex=True)
        .str.strip("_")
    )
    return df

def clean_values(df):
    df = df.copy()

    yes_map = {
        "yes", "y", "true", "1", "sometimes", "often", "i know some", "yes, i know several", "possibly"
    }
    no_map = {
        "no", "n", "false", "0", "never", "no, i don't know any"
    }

    unsure_map = {
        "i don't know", "maybe", "unsure", "don't know"
    }
    
    for col in df.columns:
        if col.startswith(("do", "does", "is", "has", "would", "have")):
            df[col] = (
                df[col]
                .astype(str)
                .str.strip()
                .str.lower()
                .replace({"nan":"", "none":""})
            )
            
            def mapper(x):
                if x == "":
                    return -1
                if x.replace(".", "", 1).isdigit():
                    if float(x) == 1.0:
                        return 1
                    elif float(x) == 0.0:
                        return 0
                    else:
                        return -1
                if x in yes_map:
                    return 1
                elif x in no_map:
                    return 0
                elif x in unsure_map:
                    return 2
                else:
                    return -1

            df[col] = df[col].apply(mapper)

    for col in df.columns:
        if "gender" in col:
            df[col] = (
                df[col]
                .astype(str)
                .str.strip()
                .str.lower()
                .replace({"nan":"", "none":""})
            )
            df[col] = df[col].apply( 
                lambda x: (1 if x.startswith("f")
                           else 0 if x.startswith("m")
                           else -1 if x == ""
                           else x)
            )
                                  
    return df


def var_mp(df):
    s = "you_feel_more_comfortable_talking_to_your_coworkers_about_your_physical_health_or_your_mental_health"
    
    df[s] = df[s].apply(
        lambda x: (1 if x == "Physical health"
                   else 2 if x == "Mental health"
                   else 0 if x == "Same level of comfort for each"
                   else -1 )   
    )
    return df

# 2017
df = pd.read_csv(r"processed/n17.csv")
df = renaming_col(df)
df = normalize_columns(df)
df = clean_values(df)
df = var_mp(df)
df.to_csv("processed/n17.csv")

# 2018
df = pd.read_csv(r"processed/n18.csv")
df = renaming_col(df)
df = normalize_columns(df)
df = clean_values(df)
df = var_mp(df)
df.to_csv("processed/n18.csv")

# 2019
df = pd.read_csv(r"processed/n19.csv")
df = renaming_col(df)
df = normalize_columns(df)
df = clean_values(df)
df = var_mp(df)
df.to_csv("processed/n19.csv")

# 2020
df = pd.read_csv(r"processed/n20.csv")
df = renaming_col(df)
df = normalize_columns(df)
df = clean_values(df)
df = var_mp(df)
df.to_csv("processed/n20.csv")

# 2021
df = pd.read_csv(r"processed/n21.csv")
df = renaming_col(df)
df = normalize_columns(df)
df = clean_values(df)
df = var_mp(df)
df.to_csv("processed/n21.csv")

In [15]:
# 80% CLEANING DONE: MERGING ALL FILES FOR THE FINAL CLEAN 

file_map = {
    r"processed/n17.csv": 2017,
    r"processed/n18.csv": 2018,
    r"processed/n19.csv": 2019,
    r"processed/n20.csv": 2020,
    r"processed/n21.csv": 2021,
}

dfs = []

for file, year in file_map.items():
    df = pd.read_csv(file)
    df["survey_year"] = year
    dfs.append(df)

merged_df = pd.concat(dfs, ignore_index=True)
merged_df.to_csv("processed/partial_clean_merge.csv")


In [22]:
# THE FINAL CLEAN: CATEGORICAL LABELS, DROPPING NULL ATTRIBUTES, OBSCURE GENDER VAL, DOCUMENTATION ETC

df = pd.read_csv(r"processed/partial_clean_merge.csv")
rows = df.shape[0]
new_columns = [
    'id1', 'id2', 'id3',
    'employee_size',
    'tech_company',
    'tech_role',
    'mh_benefits',
    'know_mh_options',
    'employer_discussed_mh',
    'mh_resources',
    'anonymity_protected',
    'comfort_talking_health',
    'comfort_talking_supervisor',
    'discussed_mh_with_employer',
    'comfort_talking_coworkers',
    'discussed_mh_with_coworkers',
    'employer_importance_physical',
    'employer_importance_mental',
    'medical_coverage',
    'know_local_resources',
    'productivity_affected',
    'work_time_affected',
    'past_mh_disorder',
    'sought_treatment',
    'family_history_mh',
    'observations_effect',
    'age',
    'gender',
    'race',
    'survey_year'
]

df.columns = new_columns


def na_percent(df): # to decide drop column needs
    for col in df.columns:
        print(f"{col}: {df[col].isna().sum()}, percentage = {(df[col].isna().sum()/rows)*100}")

del(df['work_time_affected'])

df.fillna(-1, inplace = True)      

vals = set(df['employee_size'])
count_dict = {'1-5':'1 to 5', 
              '6-25':'6 to 25', 
              '26-100': '26 to 100', 
              '100-500': '100 to 500', 
              '500-1000': '500 to 1000', 
              'More than 1000': '1000+', 
              -1: 'Not disclosed'}

df['employee_size'] = df['employee_size'].map(count_dict)

gen = set(df['gender'])

def gender_handling(x):
    if x == 0 or x == '0':
        return 'Female'
    if x == 1 or x == '1':
        return 'Male'
    if x == -1:
        return 'Non-binary / Other / Not disclosed'
    
    # handling text
    if not isinstance(x, str):
        return 'Non-binary / Other / Not disclosed'
    
    x = x.lower()
    
    # male stuff
    if any(k in x for k in [
        'male', 'man', 'dude', 'penis', 'swm',
        'cis male', 'cis-male', 'cishet male',
        'identify as male'
    ]):
        return 'Male'
    
    # Female indicators
    if any(k in x for k in [
        'female', 'woman', 'she', 'fem',
        'cis female', 'cis-female',
        'identify as female', 'trans woman',
        'trans female', 'transfeminine'
    ]):
        return 'Female'
    
    # Everything else
    return 'Non-binary / Other / Not disclosed'
    
df['gender'] = df['gender'].apply(gender_handling)

race = set(df['race'])
df.replace({'race': {'I prefer not to answer': -1, 'More than one of the above': 'Multi Race'}}, inplace = True)

def age_cleaner(x):
    if x <= 18 or x >= 90:
        return False  
    else:
        return True  

df = df[df['age'].apply(age_cleaner)]

print(df['age'])

del(df['id1'])
del(df['id2'])
del(df['id3'])

df.to_csv('final_clean.csv', index = False)

0       27.0
1       31.0
2       36.0
3       22.0
4       52.0
        ... 
1831    33.0
1832    49.0
1833    28.0
1834    26.0
1835    38.0
Name: age, Length: 1827, dtype: float64


In [162]:
df = pd.read_csv(r"C:\Users\shrin\anaconda_projects\22ad000e-f511-4225-a75f-52a005cf001a\final_cleaned.csv")
print(df["age"].describe())
print(df["employer_importance_mental"].unique())

count    1827.000000
mean       34.793651
std         8.551410
min        19.000000
25%        28.000000
50%        34.000000
75%        40.000000
max        67.000000
Name: age, dtype: float64
[ 0.  2.  1.  5. -1. 10.  8.  7.  3.  9.  6.  4.]
